# 4.3 오버피팅과 정규화 (Overfitting and Regularization)

머신러닝 모델
과적합(Overfitting)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 50
BATCH_SIZE = 64

## 데이터셋에 노이즈 추가하기

![original.png](https://github.com/yellowjs0304/3-min-pytorch/blob/master/04-%ED%8C%A8%EC%85%98_%EC%95%84%EC%9D%B4%ED%85%9C%EC%9D%84_%EA%B5%AC%EB%B6%84%ED%95%98%EB%8A%94_DNN/assets/original.png?raw=1)

![horizontalflip.png](https://github.com/yellowjs0304/3-min-pytorch/blob/master/04-%ED%8C%A8%EC%85%98_%EC%95%84%EC%9D%B4%ED%85%9C%EC%9D%84_%EA%B5%AC%EB%B6%84%ED%95%98%EB%8A%94_DNN/assets/horizontalflip.png?raw=1)

In [4]:
# Train Data Loader
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
    
# Test Data Loader    
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw


Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw


Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw


Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 뉴럴넷으로 Fashion MNIST 학습하기

입력 `x` 는 `[배치크기, 색, 높이, 넓이]`로 이루어져 있습니다.
`x.size()`를 해보면 `[64, 1, 28, 28]`이라고 표시되는 것을 보실 수 있습니다.
Fashion MNIST에서 이미지의 크기는 28 x 28, 색은 흑백으로 1 가지 입니다.
그러므로 입력 x의 총 특성값 갯수는 28 x 28 x 1, 즉 784개 입니다.

우리가 사용할 모델은 3개의 레이어를 가진 뉴럴네트워크 입니다. 

In [5]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        # 드롭아웃 확률
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = F.relu(self.fc2(x))
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = self.fc3(x)
        return x

## 모델 준비하기 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다.
일반적으로 CPU 1개만 사용할 경우 필요는 없지만,
GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다.
지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [6]:
model        = Net(dropout_p=0.2).to(DEVICE)
optimizer    = optim.SGD(model.parameters(), lr=0.01)

## 학습하기

In [7]:
def train(model, train_loader, optimizer):
    # 모델을 학습 모드로 전환
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        # 매 반복(iteration)마다 기울기를 계산하기 위해 zero_grad() 호출
        optimizer.zero_grad()
        #실제 모델의 예측값(output) 받아오기
        output = model(data)
        # 정답 데이터와의 Cross Entropy Loss 계산
        loss = F.cross_entropy(output, target)
        # 기울기 계산
        loss.backward()
        # 가중치 수정
        optimizer.step()

## 테스트하기


In [8]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()
    # 필요한 변수 초기화
    # Test과정에서의 Loss = test_loss
    # 실제 모델의 예측이 정답과 맞은 횟수 = correct
    test_loss = 0
    correct = 0
    with torch.no_grad():# 평가 과정에서는 기울기를 계산하지 않으므로, no_grad명시
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 맞춘 갯수 계산
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측 클래스(pred)과 정답 클래스를 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #정확도 계산
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 0.6525, Accuracy: 76.71%
[2] Test Loss: 0.5396, Accuracy: 80.55%
[3] Test Loss: 0.4960, Accuracy: 82.00%
[4] Test Loss: 0.4611, Accuracy: 83.26%
[5] Test Loss: 0.4465, Accuracy: 83.49%
[6] Test Loss: 0.4343, Accuracy: 83.95%
[7] Test Loss: 0.4237, Accuracy: 84.39%
[8] Test Loss: 0.4053, Accuracy: 85.36%
[9] Test Loss: 0.4056, Accuracy: 85.12%
[10] Test Loss: 0.3897, Accuracy: 85.82%
[11] Test Loss: 0.3827, Accuracy: 86.00%
[12] Test Loss: 0.3834, Accuracy: 86.18%
[13] Test Loss: 0.3761, Accuracy: 86.40%
[14] Test Loss: 0.3690, Accuracy: 86.73%
[15] Test Loss: 0.3745, Accuracy: 86.29%
[16] Test Loss: 0.3733, Accuracy: 86.20%
[17] Test Loss: 0.3567, Accuracy: 87.33%


KeyboardInterrupt: ignored